In [1]:
import pandas as pd
df = pd.read_csv('../data/Original_csv/Hyperion.csv')

In [5]:
import string

def find_word_bounds(spans: list, text: str) -> list:
    bounds = []
    end = 0
    for span in spans:
        s = span.translate(str.maketrans('', '', string.punctuation))
        word_list = s.split()
        text_list = text.translate(str.maketrans('', '', string.punctuation)).split()
        try:
            start = text_list.index(word_list[0], end)
        except:
            if not bounds:
                start = 0
            else:
                
                start = bounds[-1][1] + 1
        end = start + len(word_list) - 1
            
        bounds.append((start, end))
    return bounds

In [10]:
dataset = []

for row in df.itertuples():
    text = row.Testo
    if pd.isna(text):
        sample['Stralci'].append(row.Stralcio)
        sample['Repertori'].append(row.Repertorio)
        
    else: 
        sample = {}
        sample['Testo'] = text
        sample['Stralci'] = [row.Stralcio]
        
        sample['Repertori'] = [row.Repertorio]
        dataset.append(sample)

full_text_bounds = []       
for sample in dataset:
    sample['Bounds'] = find_word_bounds(sample['Stralci'], sample['Testo'])
    full_text_bounds.append(find_word_bounds([sample['Testo']], sample['Testo']))


In [15]:
import numpy as np
# A è B sono tupe con i bound dello span
def IoU(A, B):
    if A == B:
        return 1
    start = max(A[0], B[0])
    end = min(A[1], B[1])
    if(start > end):
        return 0
    intersection = end - start
    return intersection / (A[1] - A[0] + B[1] - B[0] - intersection)

def compute_IoUs(pred_bounds, gt_spans):
    IoUs = []
    for gt_bounds in gt_spans:
        IoUs.append(IoU(pred_bounds, gt_bounds)) 
    return IoUs

#Input: text_spans_dict = [ {
#           'Bounds' : (a,b), 
#           'IoU' : float,
#           'Repertorio': 'string':
#           } ]
def normalize(text_spans_dict, gt_spans):
    normalized = []
    for i in range(len(text_spans_dict)):
        #normalized is not empty
        if normalized:
            if normalized[-1]['Repertorio'] == text_spans_dict[i]['Repertorio']:
                new_span = (normalized[-1]['Bounds'][0], text_spans_dict[i]['Bounds'][1])
                new_span_features = {
                    'Bounds' : new_span, 
                    'IoU' : None,
                    'Repertorio' : text_spans_dict[i]['Repertorio']
                    }
                del normalized[-1]
                normalized.append(new_span_features)
            else:
                normalized.append(text_spans_dict[i])
        else:
            normalized.append(text_spans_dict[i])
        
    
    for i in range(len(normalized)):
        normalized[i]['IoU'] = max(compute_IoUs(normalized[i]['Bounds'], gt_spans['Bounds']))
    return normalized
    

metrics = []
normalized_metrics = []
for i, pred_bounds in enumerate(full_text_bounds):
    text_IoUs = []
    for pred_span in pred_bounds:
        IoUs = compute_IoUs(pred_span, dataset[i]['Bounds'])
        best = np.argmax(IoUs)
        span_features = {
            'Bounds' : pred_span, 
            'IoU' : IoUs[best],
            'Repertorio' : dataset[i]['Repertori'][best]
            }

        text_IoUs.append(span_features)
    metrics.append(text_IoUs)
    normalized_metrics.append(normalize(text_IoUs, dataset[i]))

In [16]:
print('----------------------------------------------------------')
print('Risultati labels GT e stralci non uniti')


n_spans = 0
for e in dataset:
    n_spans += len(e['Bounds'])
print('Numero stralci nel dataset:', str(n_spans))

n_spans = 0
for e in metrics:
    n_spans += len(e)
print('Numero stralci predetti:', str(n_spans))

mean = 0
long_spans = 0
min_lenght = 0
perfect_spans =0
for text in metrics:
    for span in text:
        if span['Bounds'][1] - span['Bounds'][0] >= min_lenght:
            long_spans += 1
            mean += span['IoU']
            if span['IoU'] == 1:
                perfect_spans += 1
perfect_spans_perc = perfect_spans / long_spans
mean_IoU = mean / long_spans
print('Numero stralci con lunghezza minima = ', str(min_lenght), ': ', str(long_spans))
print('Media IoU:', str(mean_IoU))
print('Percentuale span perfetti: ', str(perfect_spans_perc))

----------------------------------------------------------
Risultati labels GT e stralci non uniti
Numero stralci nel dataset: 35148
Numero stralci predetti: 15332
Numero stralci con lunghezza minima =  0 :  15332
Media IoU: 0.8221715025284804
Percentuale span perfetti:  0.6314896947560658
